# Análise exploratória de dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from IPython.display import display, Markdown

palette = sns.color_palette("pastel")
sns.set_style("whitegrid")
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette(palette))

In [ ]:
df = pd.read_csv('../data/raw/diabetes_prediction_dataset.csv')
df.head()

## 1. Descrição dos dados

![Diabetes](https://tschealth.com/wp-content/uploads/2023/06/Diabetes-Disease-1170x740.jpg)

**Diabetes prediction dataset** é um conjunto de dados médicos e demográficos de pacientes, juntamente com seu status de diabetes (positivo ou negativo). Os dados incluem características como idade, sexo, índice de massa corporal (IMC), hipertensão, doenças cardíacas, histórico de tabagismo, nível de açucar e nível de glicose no sangue. Este conjunto de dados pode ser usado para construir modelos de aprendizado de máquina para prever diabetes em pacientes com base em seu histórico médico e informações demográficas. Isto pode ser útil para os profissionais de saúde na identificação de pacientes que possam estar em risco de desenvolver diabetes e no desenvolvimento de planos de tratamento personalizados. Além disso, o conjunto de dados pode ser usado por pesquisadores para explorar as relações entre vários fatores médicos e demográficos e a probabilidade de desenvolver diabetes.

#### Dicionário dos dados:

In [ ]:
dict = Path("../data/external/dictionary.csv", index = False)
dicionario = pd.read_csv(dict, sep=',')
dicionario

#### Dados faltantes:

In [ ]:
null_counts = df.isnull().sum()

print(null_counts)

display(Markdown(
    "---\n"
    f"Sabemos assim que a não existem valores faltantes."))

In [ ]:
df.describe(include='all')

Observações:

- Mulheres são a maioria no conjunto de dados;
- A idade média dos pacientes é de aproximadamente 42 anos;
- O paciente mais jovem tinha aproximadamente 1 mês de vida;
- O paciente mais velho tinha 80 anos.

---

Vamos verificar a necessidade de manter o valor "Other" da coluna "gender"

In [ ]:
count = (df['gender'] == 'Other').sum() #conta a presença do valor "Other" na coluna "gender"
print(f"Número de linhas com 'other' na coluna 'gender': {count}")

Sendo um valor irrisório comparado à quantidade de dados nesse dataset, removeremos os valores "Other" da coluna.

In [ ]:
df = df[df['gender'] != 'Other']

---

Agora verificaremos a presença de linhas duplicadas. Considerando a variabilidade em casas decimais de algumas variáveis, podemos assumir que linhas com os mesmos valores se referem ao mesmo paciente.

In [ ]:
duplicatas: int = df.duplicated().sum()
print(f"Número de duplicatas: {duplicatas}")
df = df.drop_duplicates()
display(Markdown(
    "---\n"))

## 2. Perguntas de partida e hipóteses

- Existe relação entre diabetes e gênero?
- O historico de fumante do paciente tem relação com a diabetes?
- Qual a relação entre diabetes e glicose/açucar no sangue?
- Qual a correlação entre os dados?
- Existem dados faltantes, se sim quantos e em quais variáveis?
- Qual a distribuição dos dados (variável por variável)?
- Existe alguma relação entre o gênero, idade e o diagnóstico de diabetes nos pacientes?
- Existe relação entre as doenças de hipertenção, doença no coração e o dignóstico de diabetes?


## 3 Análises

### 3.1 Análises univariadas

In [ ]:
fig, axes = plt.subplots(2,3,figsize=(20, 10)) #cria conjunto de gráficos e define o tamanho

#historgrama da idade
ax = df['age'].hist(bins=30, ax=axes[0,0], edgecolor='black') #cria o histograma
ax.grid(False) #remove grade
ax.set_title('Histograma da idade') #adiciona título

#histograma do IMC
ax = df['bmi'].hist(bins=30, ax=axes[0,1], edgecolor='black')
ax.grid(False)
ax.set_title('Histograma do IMC')

# gráfico da distribuição de gênero
sns.countplot(x='gender',data=df, ax=axes[0,2]) #cria o gráfico
axes[0,2].set_title('Distribuição de gênero') # adiciona título
axes[0,2].set_xlabel('')  #remove a legenda inferior

# Gráfico da variável hypertension
sns.countplot(x='hypertension',data=df, ax=axes[1,0])
axes[1,0].set_title('Distribuição da variável hypertension')
axes[1,0].set_xlabel('')


# Gráfico da variável heart_disease
sns.countplot(x='heart_disease',data=df, ax=axes[1,1])
axes[1,1].set_title('Distribuição da variável heart_disease')
axes[1,1].set_xlabel('')

# Gráfico da variável smoking_history
sns.countplot(x='smoking_history',data=df, ax=axes[1,2])
axes[1,2].set_title('Distribuição da variável smoking_history')
axes[1,2].set_xlabel('')

#### Observações:

- A grande maioria dos paciente tem cerca de 80 anos;

- A grande maioria dos pacientes possui IMC em torno de 30;

- A maioria dos pacientes é composta por mulheres;

- A grande maioria dos pacientes observados não possuia hipertensão ou doeça do coração, porém hipertensão é mais frequente;

- A grande maioria não tem histórico de tabagismo.

In [ ]:
#cria gráfico relacionando genero e diabetes
plt.figure(figsize=(14,7))
df_diabetes = df[df['diabetes'] == 1]
palette = {'Female': 'pink', 'Male': 'cornflowerblue'}
sns.countplot(x='gender',data=df_diabetes,palette=palette)
plt.show()

---

- A quantidade de mulheres diabéticas é levemente superior a de homens.

---

### 3.2 Análise bivariada

In [ ]:
# Gráficos de dispersão entre variáveis numéricas
num_vars = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level','diabetes'] #seleciona apenas as variáveis numéricas
sns.pairplot(df[num_vars], hue='diabetes') #cria uma matriz de gráficos de dispersão
plt.suptitle('Gráficos de dispersão entre as variáveis numéricas',y=1.02)
plt.show()

#### Observações:

- Todos os indivíduos com o nível de glicose no sangue acima de 200 são considerados diabéticos;
- Dos diabéticos que registraram glicose abaixo de 200, a maioria é composta por indívidos acima de 50 anos;
- Indivíduos com idade superior a 30 anos aparentam estar mais suscetíveis a apresentar diabetes mesmo com IMC inferior a 40;
- Observamos indivíduos diabéticos com glicose controlada porém com níveis de hemoglobina glicada altos. 

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(15, 10)) #cria conjunto de gráficos e define o tamanho


#boxplot de diabetes por idade
sns.boxplot(x='diabetes', y='age', data=df, palette='Set2',ax = axes[0,0]) #cria o gráfico
axes[0,0].set_title('Distribuição de diabetes por idade') #adiciona título
axes[0,0].set_xlabel('') #remove legenda inferior
axes[0,0].set_ylabel('Idade')

#boxplot de diabetes por IMC
sns.boxplot(x='diabetes',y='bmi', data=df, palette='Set2',ax = axes[0,1])
axes[0,1].set_title('Distribuição de diabetes por IMC')
axes[0,1].set_xlabel('')
axes[0,1].set_ylabel('IMC')

#boxplot de HbA1c por diabetes
sns.boxplot(x='diabetes',y='HbA1c_level', data=df, palette='Set2',ax = axes[1,1])
axes[1,1].set_title('Distribuição de diabetes por HbA1c')
axes[1,1].set_xlabel('')
axes[1,1].set_ylabel('HbA1c')

#boxplot de diabetes por glicose no sangue
sns.boxplot(x='diabetes',y='blood_glucose_level', data=df, palette='Set2',ax = axes[1,0])
axes[1,0].set_title('Distribuição de diabetes por glicose no sangue')
axes[1,0].set_xlabel('')
axes[1,0].set_ylabel('glicose no sangue')

#### Observações:

- Pacientes com maior IMC aparentam estar mais propícios a apresentar diabetes;

- Todos os pacientes diabéticos tinham glicose no sangue acima de 100;

- Muitos pacientes com níveis de glicose e hemoglobina glicada altas não foram considerados diabéticos;

- Os níveis de hemoglobina glicada em diabéticos foram mais concentrados entre 6% e 8%;

### 3.3 Análises multivariadas

In [ ]:
#boxplots do IMC por histórico de tabagismo 
plt.figure(figsize=(12, 8)) #cria figura
sns.boxplot(x='smoking_history', y='bmi', hue='diabetes', data=df[df['smoking_history'] != 'No Info'], palette='Set2') #Cria o gráfico sem o valor "no info" para a variável smoking_history
plt.title('Distribuição de IMC por histórico de tabagismo e diabetes')
plt.xlabel('Histórico de tabagismo') 
plt.ylabel('IMC')
plt.show()

In [ ]:
#boxplots de IMC por gênero por diabetes
plt.figure(figsize=(12, 8))
pal_gen = dict(Female='pink', Male='cornflowerblue') #cria um dicionário para servir como paleta
sns.boxplot(x='diabetes', y='bmi', hue='gender', data=df, palette=pal_gen)
plt.title('Distribuição de IMC por gênero por diabetes')
plt.xlabel('Diabetes')
plt.ylabel('IMC')

### Observações:

- Não parece haver relação entre tabagismo e o diagnóstico de diabetes;
- Mulheres diabéticas apresentaram IMC levemente maior que dos homens diabéticos, com maior variabilidade e limite superior maior.

### 3.4 Correlação

In [ ]:
var_quantitativas = df.select_dtypes(include=['int','float']) #seleciona as variáveis quantitativas
matriz_correlacao = var_quantitativas.corr() #cria a matriz de correlação
plt.figure(figsize=(15, 10))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm',center=0, fmt=".2f") #cria o heatmap para visualização da matriz
plt.title('Matriz de correlação')
plt.show()

#### Observações:

- Os maiores índices de correlação com a diabetes são, respectivamente, blood_glucose_level com 42% e HbA1c_level com 40%;
- Os menores índices de correlação com a diabetes são, respectivamente, heart_disease com 17% e hypertension com 20%;
- Moderada correlação entre a idade e o diagnóstico de diabetes;
- Observamos também correlação significativa entre o IMC do paciente e o diagnóstico de diabetes.

## 3. *Insights*

#### Com essas análises obtivemos os seguintes insights:

- Apesar de termos mais mulheres diabéticas do que homens diabéticos proporcionalmente, temos 7,61% de mulheres diabéticas e 9,75% de homens diabéticos;

- Não observamos relação significativa entre o histórico de tabagismo e o dignóstico de diabetes;

- Todos os indivíduos com o nível de glicose no sangue acima de 200 são considerados diabéticos;

- As maiores correlações associadas ao diagnóstico positivo de diabetes foram das variáveis nível de glicose, hemoglobina glicada, idade e IMC, respectivamente;

- Mulheres diabéticas apresentaram IMC levemente maior que dos homens diabéticos, com maior variabilidade e limite superior maior;

- Indivíduos com idade superior a 30 anos aparentam estar mais suscetíveis a apresentar diabetes.

- Não foram constatadas relações significativas entre doenças do coração e hipertenção quanto ao diagnóstico de diabetes.